## Siamese NN

In [1]:
import os
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from SNN.siamese_network import *
from SNN.siamese_network_parse import PrepareDataSNN
from sklearn.model_selection import train_test_split


2025-01-08 23:38:39.906781: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-08 23:38:40.612364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Preparar la data

In [2]:
data = PrepareDataSNN()

# Ruta para guardar los datos
data_dir = "./data_snn"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de entrenamiento y validación desde archivo...")
    data = np.load(data_file)
    X1_train, X1_val = data["X1_train"], data["X1_val"]
    X2_train, X2_val = data["X2_train"], data["X2_val"]
    y_train, y_val = data["y_train"], data["y_val"]
else:
    # Crear los conjuntos de datos y guardarlos
    print("Dividiendo los datos y guardándolos...")
    data_a, data_b, labels = PrepareDataSNN().process()
    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
        data_a, data_b, labels, test_size=0.2, random_state=42
    )
    np.savez(data_file, X1_train=X1_train, X1_val=X1_val, 
                X2_train=X2_train, X2_val=X2_val, 
                y_train=y_train, y_val=y_val)

# Verificar las formas de los arrays
print("Formas de los conjuntos de entrenamiento:")
print(f"X1_train: {X1_train.shape}")
print(f"X2_train: {X2_train.shape}")
print(f"y_train: {y_train.shape}")

print("\nFormas de los conjuntos de validación:")
print(f"X1_val: {X1_val.shape}")
print(f"X2_val: {X2_val.shape}")
print(f"y_val: {y_val.shape}")


Cargando conjuntos de entrenamiento y validación desde archivo...
Formas de los conjuntos de entrenamiento:
X1_train: (720, 65)
X2_train: (720, 65)
y_train: (720,)

Formas de los conjuntos de validación:
X1_val: (180, 65)
X2_val: (180, 65)
y_val: (180,)


## Inicializar la red siamesa, entrenar y guardar el modelo

In [3]:
import keras
# Inicializar y entrenar la red
input_shape = (65,)  # Asumiendo que tus vectores de características tienen 65 elementos
siamese_net = SiameseNeuralNetwork(input_shape)
history = siamese_net.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val))

# Guardar el modelo
siamese_net._save_model("model_1")
# keras.saving.save_model(siamese_net, './models/a.keras')

Epoch 1/20


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5399 - loss: 0.7460 - val_accuracy: 0.6333 - val_loss: 0.6386
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5872 - loss: 0.6708 - val_accuracy: 0.7778 - val_loss: 0.5775
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7337 - loss: 0.5734 - val_accuracy: 0.7778 - val_loss: 0.5278
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7949 - loss: 0.4993 - val_accuracy: 0.7556 - val_loss: 0.5155
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8449 - loss: 0.4170 - val_accuracy: 0.7611 - val_loss: 0.5080
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8605 - loss: 0.3788 - val_accuracy: 0.7389 - val_loss: 0.5742
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8737 - loss: 0.3662 - val_accuracy: 0.7556 - val_loss: 0.5672
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8407 - loss: 0.3627 - val_accuracy: 0.7611 - val_loss: 0.5692
Epo

## Cargar el modelo guardado

In [10]:
model = load_model('./models/best_model_1-3_dataset.keras', custom_objects={'L1Distance': L1Distance})

model.summary()

Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_14      │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_4        │ (None, 128)       │    198,016 │ input_layer_13[0… │
│ (Sequential)        │                   │            │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance_4       │ (None, 128)       │          0 │ sequential_4[0][… │
│ (L1Distance)        │                   │            │ sequential_4[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 1)         │        129 │ l1_distance_4[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 594,437 (2.27 MB)

 Trainable params: 198,145 (774.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 396,292 (1.51 MB)

## Preparar la nueva data de validacion (Proyectos de C#)

In [6]:

# Ruta para guardar los datos
data_dir = "./data_val"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de validación desde archivo...")
    data = np.load(data_file)
    X1_new, X2_new, y_new = data["X1_new"], data["X2_new"], data["y_new"]
else:
    print("Procesando los datos de validación y guardándolos...")
    data_val = PrepareDataSNN("data/features_vect_val/")
    X1_new, X2_new, y_new = data_val.process()
    
    # Guardar los datos procesados
    np.savez(data_file, X1_new=X1_new, X2_new=X2_new, y_new=y_new)


# Verificar las formas de los arrays
print("Formas de los conjuntos de validación:")
print(f"X1_new: {X1_new.shape}")
print(f"X2_new: {X2_new.shape}")
print(f"y_new: {y_new.shape}")


Cargando conjuntos de validación desde archivo...
Formas de los conjuntos de validación:
X1_new: (116, 65)
X2_new: (116, 65)
y_new: (116,)


## Evaluar el modelo con los datos nuevos

In [17]:
m = model.evaluate([X1_new, X2_new], y_new)

predictions = (model.predict([X1_new, X2_new]) > 0.5).astype(int).flatten()
        
# Calcular la matriz de confusión
tn, fp, fn, tp = confusion_matrix(y_new, predictions).ravel()

# Calcular métricas adicionales
precision = precision_score(y_new, predictions)
recall = recall_score(y_new, predictions)
f1 = f1_score(y_new, predictions)

# Imprimir las métricas adicionales
print(f"Verdaderos Positivos (TP): {tp}")
print(f"Falsos Positivos (FP): {fp}")
print(f"Verdaderos Negativos (TN): {tn}")
print(f"Falsos Negativos (FN): {fn}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Exhaustividad (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7708 - loss: 0.4446 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step
Verdaderos Positivos (TP): 8
Falsos Positivos (FP): 5
Verdaderos Negativos (TN): 82
Falsos Negativos (FN): 21
Precisión (Precision): 0.6154
Exhaustividad (Recall): 0.2759
F1-Score: 0.3810


In [18]:
metrics = siamese_net.evaluate(X1_new, X2_new, y_new)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7167 - loss: 0.4662 
Pérdida en los nuevos datos: 0.4435
Accuracy en los nuevos datos: 0.7241
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Verdaderos Positivos (TP): 8
Falsos Positivos (FP): 11
Verdaderos Negativos (TN): 76
Falsos Negativos (FN): 21
Precisión (Precision): 0.4211
Exhaustividad (Recall): 0.2759
F1-Score: 0.3333


## Hacer una predicción

In [12]:
code1_features = np.random.rand(65)  # Ejemplo de vector de características
code2_features = np.random.rand(65)  # Ejemplo de vector de características

similarity = siamese_net.predict_similarity(code1_features, code2_features)

# Interpretar el resultado
if similarity < 0.5:
    print(f"Los códigos son similares con una probabilidad de {1-similarity:.2f}")
else:
    print(f"Los códigos son diferentes con una probabilidad de {similarity:.2f}")

print(f"La similitud entre los dos códigos es: {similarity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Los códigos son similares con una probabilidad de 0.57
La similitud entre los dos códigos es: 0.43302974104881287


## Buscando la mejor semilla para el modelo

In [9]:
import os
import tensorflow as tf
import numpy as np
import random

# Definir un rango de semillas
SEEDS = range(50)  # Probar con semillas del 0 al 49

# Variables para almacenar los mejores resultados
best_accuracy = 0
best_precision = 0
best_seed = None
best_model = None

# Datos de entrenamiento y validación
# Asegúrate de tener `X1_train`, `X2_train`, `y_train`, `X1_val`, `X2_val`, `y_val` ya definidos.

for seed in SEEDS:
    print(f"Probando con semilla: {seed}")
    
    # Configurar la semilla
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    # Crear y entrenar el modelo
    snn = SiameseNeuralNetwork(input_shape=(65,), learning_rate=0.001)
    snn.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val), epochs=10, batch_size=32)
    
    # Evaluar el modelo
    metrics = snn.evaluate(X1_new, X2_new, y_new)
    accuracy = metrics["accuracy"]
    precision = metrics["precision"]
    
    # Comparar con los mejores resultados
    if accuracy > best_accuracy or (accuracy == best_accuracy and precision > best_precision):
        best_accuracy = accuracy
        best_precision = precision
        best_seed = seed
        best_model = snn.model  # Guardar el modelo actual como el mejor

# Imprimir los resultados
print(f"Mejor semilla: {best_seed}")
print(f"Mejor accuracy: {best_accuracy:.4f}")
print(f"Mejor precisión: {best_precision:.4f}")

# Guardar el mejor modelo
tf.keras.config.enable_unsafe_deserialization()
best_model.save("models/best_model_1-3_dataset.keras")


Probando con semilla: 0
Epoch 1/10


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4856 - loss: 0.7090 - val_accuracy: 0.6889 - val_loss: 0.6646
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5702 - loss: 0.6910 - val_accuracy: 0.7222 - val_loss: 0.6273
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6446 - loss: 0.6488 - val_accuracy: 0.7389 - val_loss: 0.5802
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7385 - loss: 0.5570 - val_accuracy: 0.7722 - val_loss: 0.5317
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8272 - loss: 0.4587 - val_accuracy: 0.7333 - val_loss: 0.5445
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8311 - loss: 0.4163 - val_accuracy: 0.7500 - val_loss: 0.5470
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8443 - loss: 0.3832 - val_accuracy: 0.7611 - val_loss: 0.5607
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8475 - loss: 0.3564 - val_accuracy: 0.7611 - val_loss: 0.5791
Epo

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5076 - loss: 0.7342 - val_accuracy: 0.6000 - val_loss: 0.6548
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5516 - loss: 0.6900 - val_accuracy: 0.6778 - val_loss: 0.6231
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6915 - loss: 0.6179 - val_accuracy: 0.7056 - val_loss: 0.5696
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7643 - loss: 0.5388 - val_accuracy: 0.7389 - val_loss: 0.5349
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8373 - loss: 0.4482 - val_accuracy: 0.7500 - val_loss: 0.5307
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8865 - loss: 0.3459 - val_accuracy: 0.7167 - val_loss: 0.5726
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8549 - loss: 0.3879 - val_accuracy: 0.7667 - val_loss: 0.5395
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8656 - loss: 0.3282 - val_accuracy: 0.7611 - val_loss: 0.5703
Epo

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5184 - loss: 0.7231 - val_accuracy: 0.6944 - val_loss: 0.6437
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6126 - loss: 0.6526 - val_accuracy: 0.7833 - val_loss: 0.5856
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6972 - loss: 0.5889 - val_accuracy: 0.8222 - val_loss: 0.5423
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7727 - loss: 0.5005 - val_accuracy: 0.7778 - val_loss: 0.5332
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8486 - loss: 0.3945 - val_accuracy: 0.7556 - val_loss: 0.5449
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8667 - loss: 0.3482 - val_accuracy: 0.7556 - val_loss: 0.5624
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8736 - loss: 0.3447 - val_accuracy: 0.7778 - val_loss: 0.5641
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8513 - loss: 0.3468 - val_accuracy: 0.7722 - val_loss: 0.6200
Epo

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5159 - loss: 0.7224 - val_accuracy: 0.5722 - val_loss: 0.6732
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5855 - loss: 0.6643 - val_accuracy: 0.6611 - val_loss: 0.6540
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6096 - loss: 0.6517 - val_accuracy: 0.7278 - val_loss: 0.6189
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7027 - loss: 0.5894 - val_accuracy: 0.7500 - val_loss: 0.5818
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7975 - loss: 0.5008 - val_accuracy: 0.7500 - val_loss: 0.5564
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8283 - loss: 0.4306 - val_accuracy: 0.7667 - val_loss: 0.5648
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8482 - loss: 0.3860 - val_accuracy: 0.7611 - val_loss: 0.5820
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8781 - loss: 0.3274 - val_accuracy: 0.7611 - val_loss: 0.6155
Epo

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5399 - loss: 0.7460 - val_accuracy: 0.6333 - val_loss: 0.6386
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5872 - loss: 0.6708 - val_accuracy: 0.7778 - val_loss: 0.5775
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7337 - loss: 0.5734 - val_accuracy: 0.7778 - val_loss: 0.5278
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7949 - loss: 0.4993 - val_accuracy: 0.7556 - val_loss: 0.5155
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8449 - loss: 0.4170 - val_accuracy: 0.7611 - val_loss: 0.5080
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8605 - loss: 0.3788 - val_accuracy: 0.7389 - val_loss: 0.5742
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8737 - loss: 0.3662 - val_accuracy: 0.7556 - val_loss: 0.5672
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8407 - loss: 0.3627 - val_accuracy: 0.7611 - val_loss: 0.5692
Epo